In [6]:
# importer les packages 
import pandas as pd 
from requests import get 
from bs4 import BeautifulSoup as bs

In [7]:
# url de la page 
url = 'https://sn.coinafrique.com/categorie/chaussures-enfants'
res = get(url)
soup = bs(res.text, 'html.parser')
containers = soup.find_all('div', class_ ='col s6 m4 l3')
len(containers)

84

In [14]:
# scraper les données sur un container 
container = containers[0]
type_chaussures = container.find('p', class_  ='ad__card-description').text.strip()

prix = container.find('p', class_  ="ad__card-price").text.strip().replace('CFA', '').replace(' ','').replace(',','.')
adresse=container.find('p',class_='ad__card-location').text.strip().replace('location_on','')
img_link = container.find('a', class_ = 'card-image ad__card-image waves-block waves-light').img['src']



In [15]:
# scraper les données sur une page 
data = []
for container in containers: 
    try: 
    
        type_chaussures = container.find('p', class_  ='ad__card-description').text.strip()
        prix = container.find('p', class_  ='ad__card-price').text.strip().replace('CFA', '').replace(' ','')
        adresse=container.find('p',class_='ad__card-location').text.strip().replace('location_on','')
        img_link = container.find('a', class_ = 'card-image ad__card-image waves-block waves-light').img['src']
      
        dic = {
            'type habits': type_habits, 
            'prix':prix, 
            'adresse':adresse,
            'image lien': img_link 
        }
        data.append(dic)
    except: 
        pass

DF = pd.DataFrame(data)

In [16]:
# scraper les données sur plusieurs 
df = pd.DataFrame()
for i in range(1,9): 
    url = f'https://sn.coinafrique.com/categorie/chaussures-enfants?page={i}'
    res = get(url)
    soup = bs(res.text, 'html.parser')
    containers = soup.find_all('div', class_ ='col s6 m4 l3')
    # scraper les données sur une page 
    data = []
    for container in containers: 
        try: 
            type_chaussures = container.find('p', class_  ='ad__card-description').text.strip()
            prix = container.find('p', class_  ='ad__card-price').text.strip().replace('CFA', '')
            adresse=container.find('p',class_='ad__card-location').text.strip().replace('location_on','')
            img_link = container.find('a', class_ = 'card-image ad__card-image waves-block waves-light').img['src']
        
            dic = {
            'type chaussures': type_chaussures, 
            'prix':prix, 
            'adresse':adresse,
            'image lien': img_link 
            }
            data.append(dic)
        except: 
                pass
    
    DF = pd.DataFrame(data)
    df = pd.concat([df,DF], axis = 0).reset_index(drop = True)

In [17]:
df.shape

(627, 4)

In [18]:
df.head(10)

,type chaussures,prix,adresse,image lien
0,Chaussures enfant,10000,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5181147_u...
1,Chaussures enfant,8500,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5181134_u...
2,Chaussures enfant,7000,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5181139_u...
3,Ballerines enfants,12000,"Dakar, Sénégal",https://images.coinafrique.com/thumb_5141832_u...
4,Chaussures enfants,2000,"Dakar Plateau, Dakar, Sénégal",https://images.coinafrique.com/thumb_5125946_u...
5,Baskets enfant,3500,"Parcelle Assainies, Dakar, Sénégal",https://images.coinafrique.com/thumb_5080430_u...
6,Basket,5000,"Parcelle Assainies, Dakar, Sénégal",https://images.coinafrique.com/thumb_5080428_u...
7,Claquettes,8450,"Fass, Dakar, Sénégal",https://images.coinafrique.com/thumb_5047806_u...
8,Ensemble chaussures et chaussettes pour bebe,1800,"Dakar Plateau, Dakar, Sénégal",https://images.coinafrique.com/thumb_5037805_u...
9,Ballerines enfants,Prixsurdemande,"Dakar, Sénégal",https://images.coinafrique.com/thumb_5002639_u...


In [8]:
df.isnull().sum()

type chaussures    0
prix               0
adresse            0
image lien         0
dtype: int64

In [8]:
df['prix'] = pd.to_numeric(df['prix'], errors='coerce')#conversion de 'prix sur demande' en float
df.prix=df.prix.astype('float64')
df.dtypes

type chaussures     object
prix               float64
adresse             object
image lien          object
dtype: object

In [12]:
for col in df.columns:
    if df[col].dtypes == 'float':
        df[col].fillna(df[col].median(),inplace=True)

df.isnull().sum()

C:\Users\USER\AppData\Local\Temp\ipykernel_1076\4008989335.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(),inplace=True)


type chaussures    0
prix               0
adresse            0
image lien         0
dtype: int64

In [14]:
df.duplicated().sum()

np.int64(0)

In [13]:
df.to_csv('chaussures_enfants_url2.csv', index=False)